# Lab 8 - Transactions and Gas

## 1. Setup

### Step 1: Copy content of lab-7 into lab-8

In [ ]:
cp -rp ~/github/suss/fin579/lab-7/. ~/github/suss/fin579/lab-8
cd ~/github/suss/fin579/lab-8


## Lab 8a : Minimum Transaction Cost

### Step 1. Start HH console

```sh
hh console
```

### Step 2. Send 1 wei to accounts[1].

```js
> response = await accounts[0].sendTransaction(
    {
        to: accounts[1].address,
        value:1
    }
)
> receipt = await response.wait();

```

### Step 3. Analyse the result

* The transaction response indicates that the gasPrice was set at `1875000000`. This settings is configured in hardhat.config.js but can be overridden by passing an options object at the send of sendTraction().
* Once the transaction is mined, it returns a receipt which contains the gasUsed - `21000`.

```js
> response
{
    hash: '0x5266cf8d88c1f8fd171b4a0eb1e8869344fa781a8b127c8a1060930e1f68fe08',
    from: '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
    gasPrice: BigNumber { value: "1875000000" },
    to: '0x70997970C51812dc3A010C7d01b50e0d17dc79C8',
    value: BigNumber { value: "1" },
    ...
}
> receipt
{
    gasUsed: BigNumber { value: "21000" },
    ...
}

```


## Lab 8b : Transfer ETH with GasPrice

This is an extension of the previous example on **Transfer ETH** by overriding the send transaction with an explicit gasPrice.

Sends 1 wei from accounts[0] to accounts[1] with a gasPrice of 10 gwei.

```js
> response = await accounts[0].sendTransaction(
    {
        to: accounts[1].address,
        value:1,
        gasPrice: ethers.utils.parseUnits("10","gwei")
    }
)
> response.gasPrice
//BigNumber { value: "10000000000" }
```

## Lab 8c: Estimate Cost of Deployment

### Step 1.  Create `testGasCost.js`

NOTE: After the installation of OpenZeppelin there will be a conflict in the contract name between the ERC20.sol in OpenZeppelin and the one in contracts directory. To resolve this, you will use a fully qualified contract name "contracts/ERC20.sol:ERC20".

```js
const { expect } = require("chai");
describe("Test Gas Cost", () => {
    let accounts, factory, name, symbol, totalSupply;

    beforeEach(async () => {
        accounts = await ethers.getSigners();
        factory = await ethers.getContractFactory("contracts/ERC20.sol:ERC20");
        name = "TUT"; // <=== replace with your own token name
        symbol = "TUT"; // <=== replace with your own token symbol
        totalSupply = ethers.utils.parseUnits("1000", "ether"); // <=== replace with your own supply
    });

    
});
```

### Step 2. Test Contract Deployment Cost

```js
    it("Test Contract Deployment Cost", async () => {
        let deployed = await factory.getDeployTransaction(
            name,
            symbol,
            totalSupply,
            accounts[0].address
        );
        let gastimate = await ethers.provider.estimateGas(deployed.data);
        console.log(`Contract Deployment Const: ${gastimate}`);
    });
```

In [ ]:
hh test test/testGasCost

### Step 2.  Test token transfer cost

```js
    it("Test token transfer cost", async () => {
        let contract = await factory.deploy(
            name,
            symbol,
            totalSupply,
            account.address
        );
        let transferCost = await contract.estimateGas.transfer(
            accounts[1].address,
            1
        );
        console.log(`Token transfer cost: ${transferCost}`);
    });
```

In [ ]:
hh test test/testGasCost

## Lab 8d: Cost of Reading from Storage Data Location

In this session, we will study the cost of accessing storage data location. This example is to illustrate the cost of reading from contract storage. When calling constant functions `view` and `pure` from ethers.js, there is no gas involved because the result is obtained from the local node and is not broadcasted onto the blockchain. However, if this contract is being called by another on-chain transaction, then the cost will be incurred for reading.

### Step 1. Test cost of calling `decimals()`.

```js
    it("Test cost of calling decimal", async () => {
        let factory = await ethers.getContractFactory(
            "contracts/ERC20.sol:ERC20"
        );
        let contract = await factory.deploy(
            name,
            symbol,
            totalSupply,
            accounts[0].address
        );
        let before = await ethers.provider.getBalance(accounts[0].address);

        let pureCost = await contract.estimateGas.decimals();
        console.log(`Cost of calling decimal: ${pureCost}`);

        let after = await ethers.provider.getBalance(accounts[0].address);
        console.log("actual deduction:", before.sub(after));
    });
```

You can see from the result below that the estimated cost of calling decimals is 21407 but this cost is only applicable when invoked by contracts. When called externally, the cost is actually zero as proven by the actual deduction of 0.


In [ ]:
hh test test/testGasCost

### Step 2. Test cost of calling `balanceOf()`.

```js
    it("Test cost of calling balanceOf", async () => {
        let factory = await ethers.getContractFactory(
            "contracts/ERC20.sol:ERC20"
        );
        let contract = await factory.deploy(
            name,
            symbol,
            totalSupply,
            accounts[0].address
        );
        let before = await ethers.provider.getBalance(accounts[0].address);

        let roCost = await contract.estimateGas.balanceOf(accounts[0].address);
        console.log(`Cost of calling balanceOf: ${roCost}`);

        let after = await ethers.provider.getBalance(accounts[0].address);
        console.log("actual deduction:", before.sub(after));
    });
```

Here, we are testing the cost of reading. Similar to the above test, it cost 0 when called externally. However, since the function balanceOf() is not a `pure` function like decimals(). It costs slightly higher than decimals(). But still significantly cheaper than making a state changing call like transfer().

In [ ]:
hh test test/testGasCost